<a href="https://colab.research.google.com/github/numagic/lumos/blob/update_colab/tutorials/colab/Laptime_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Usually it is recommended to use lumos with the docker image it provides. But we can also run `lumos` with conda environment.

Google Colab provides free GPU and TPU VMs that one could use with jupyter notebook style UI, and this is what we're going to use.

To set up the environment, we'll:
1) install conda on google colab using `condacolab`
2) clone the `lumos` git repo, and setup the conda environment (this will be replaced by pip install in the future)
3) run laptime simulation example

# Install Conda on Google Colab

<!-- By Jaime Rodríguez-Guerra <@jaimergp>. Last modified 2021.08.04 -->

`condacolab` simplifies the setup as much as possible, but there are some gotchas.

**⚠️ Read this before continuing!**

* The `condacolab` commands need to be run the first Code cell!
* Once you run `condacolab.install()`, the Python kernel will be restarted. This is **normal and expected**. After that, you can continue running the cells below like normal.
* Do not use the `Run all` option. Run the `condacolab` cell _individually_ and wait for the kernel to restart. **Only then**, you can run all cells if you want.
* You can only use the `base` environment. Do not try to create new ones; instead update `base` with either:
  * `conda install <packages>`
  * `conda env update -n base -f environment.yml`
* If you want to use GPUs, make sure you are using such an instance before starting!
* If you get an error, please raise an issue [here](https://github.com/jaimergp/condacolab/issues).

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:21
🔁 Restarting kernel...


In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
# Make sure cuda is available
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Wed May 11 19:02:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |            

# Setup `lumos` environment

In [3]:
# Clone repo and enter the right path
!git clone https://github.com/numagic/lumos.git
%cd lumos
!ls

Cloning into 'lumos'...
remote: Enumerating objects: 426, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 426 (delta 67), reused 48 (delta 32), pack-reused 272
Receiving objects: 100% (426/426), 706.20 KiB | 6.60 MiB/s, done.
Resolving deltas: 100% (128/128), done.
/content/lumos
data			environment.yml  lumos		   setup.py
docker			examples	 pyproject.toml    tests
docker-compose.gpu.yml	imgs		 README.md	   tutorials
docker-compose.yml	LICENSE		 regression_tests


In [4]:
# Colab by default uses python 3.7, which we can't change. Condacolab also only
# supports the base env, which we update.
# It may take 4-5 minutes to set up the conda environment, particularly
# with setting up the cuda toolkit for the GPU.
# !conda env update -n base -f environment.yml

# Or... we could direclty install them which seems faster than asking conda to
# solve for the environment. So we'll take this to make the colab experience
# better.
# TODO: make dependency automatic -> this would require conda as there are
# non-python dependencies
!conda install -c conda-forge cyipopt
!pip install casadi pyarrow pandas
# Install the GPU version of jax to use GPU (need correct cuda version)
!pip install jax[cuda11_cudnn82] -f https://storage.googleapis.com/jax-releases/jax_releases.html
# install lumos
!pip install numagic-lumos==0.0.2rc2

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cyipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-mp-3.1.0              |    h2cc385e_1006         1.1 MB  conda-forge
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_2         145 KB  conda-forge
    conda-4.12.0               |   py37h89c1867_0         1.0 MB  conda-forge
    cyipopt-1.1.0              |   py37h4f1d67c_1         144 KB  conda-forge
    future-0.18.2              |   py37h89c1867_5         713 KB  conda-forge
    ipopt-3.14.6               |       h630875f_0         1.1 MB  conda-forge
    li

Looking in links: https://storage.googleapis.com/jax-releases/jax_releases.html
     |████████████████████████████████| 939 kB 7.0 MB/s 
     |████████████████████████████████| 126 kB 4.8 MB/s 
     |████████████████████████████████| 65 kB 4.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 128.0 MB 25 kB/s 
  Created wheel for jax: filename=jax-0.3.10-py3-none-any.whl size=1088071 sha256=0a9cc5b951fc509ade99464179cdc2945246388dc406568a9c7dc2570c4f0ade
  Stored in directory: /root/.cache/pip/wheels/b6/49/8a/c5c6fe6cbf20826b70f32ca35725228dc252aacc2ba242e190
Successfully built jax
     |████████████████████████████████| 105 kB 4.9 MB/s 


In [5]:
# test jax on GPU, if you see a spike in the GPU ram used -> yes, you're using GPU
# If you see warnings about GPU not found, then either the VM connected has no
# GPU or the support packages are not installed correctly
import jax
import jax.numpy as jnp
import numpy as np
a = np.random.randn(100, 100)
b = np.random.randn(100, 100)
c = jnp.dot(a, b)


print(jax.devices())
del a, b, c

[GpuDevice(id=0, process_index=0)]


# Run laptime simulation example

Note that unfortunately colab does not show the stdout printed to the terminal, therefore the user must use the command tabs: 'Runtime' -> 'View runtime logs' to see the stdout outputs, such as those from IPOPT.

In [6]:
import logging
import sys

use_gpu_with_jax = True
is_cyclic = True
backend = "jax" # supports jax and casadi

def main():
    import jax
    import os
    import cyipopt
    cyipopt.set_logging_level(logging.WARN)

    from lumos.models.composition import ModelMaker
    from lumos.models.simple_vehicle_on_track import SimpleVehicleOnTrack
    from lumos.models.tires.utils import create_params_from_tir_file
    from lumos.simulations.laptime_simulation import LaptimeSimulation

    TRACK_DIR = "data/tracks"

    
    # Usually GPUs are designed to operate with float32 or even float16, and are
    # much slower with doubles (float64). Here we stick with float64 to ensure
    # we have the same results as 64bit as with casadi backend.
    if use_gpu_with_jax:
        jax.config.update('jax_platform_name', 'gpu')
        os.environ['JAX_PLATFORM_NAME'] = 'GPU'
        jax.config.update("jax_enable_x64", True)
    else:
        # somehow jax doesn't see the cpu device on colab?!
        jax.config.update('jax_platform_name', 'cpu')
        os.environ['JAX_PLATFORM_NAME'] = 'CPU'
        jax.config.update("jax_enable_x64", True)


    track = "Catalunya"
    track_file = os.path.join(TRACK_DIR, track + ".csv")

    model_config = SimpleVehicleOnTrack.get_recursive_default_model_config()

    # EXAMPLE: change tire model
    # model_config.replace_subtree("vehicle.tire", ModelMaker.make_config("PerantoniTire"))

    # EXMAPLE: change an aero model
    # model_config.replace_subtree("vehicle.aero", ModelMaker.make_config("MLPAero"))

    model = SimpleVehicleOnTrack(model_config=model_config)
    params = model.get_recursive_default_params()

    # Example of changing model parameters
    # TODO: an issue here is that we need to instantiate the model first to get params
    # but that's unavoidable because without the model, we don't even know the tree
    # structure of all the submodels, let alone the default parameters.
    # params.set_param("vehicle.vehicle_mass", 1700)

    # Example: change tire parameters
    sharpened_params = create_params_from_tir_file("data/tires/sharpened.tir")
    # FIXME: here we're using private methods. We should probably add a method to change
    # the parameters of an entire node in the ParameterTree
    tire_params = params._get_subtree("vehicle.tire")
    tire_params._data = sharpened_params
    params.replace_subtree("vehicle.tire", tire_params)

    final_outputs = {}
    final_states = {}
    ocp = LaptimeSimulation(
        model_params=params,
        model_config=model_config,
        sim_config=LaptimeSimulation.get_sim_config(
            num_intervals=2500,
            hessian_approximation="exact",
            is_cyclic=is_cyclic,
            is_condensed=False,
            backend=backend,
            track=track_file,
            transcription="LGR",
        ),
    )

    x0 = ocp.get_init_guess()


    print("starting the first solve!")
    solution, info = ocp.solve(
        x0,
        max_iter=200,
        print_level=5,
        print_timing_statistics="yes",
        print_info_string="yes",
        dual_inf_tol=1e-3,
        constr_viol_tol=1e-3,
    )
    total_time = ocp.dec_var_operator.get_var(
        solution, group="states", name="time", stage=-1
    )
    print(info["status_msg"])
    print(f"finished in {info['num_iter']} iterations")
    print(f"Maneuver time {total_time:.3f} sec")

    # # We can change the parameters and solve again
    # ocp.modify_model_param("vehicle.vehicle_mass", 2100.0)
    # print("starting the second solve!")
    # solution, info = ocp.solve(
    #     solution,
    #     max_iter=200,
    #     print_level=5,
    #     print_timing_statistics="yes",
    #     print_info_string="yes",
    #     derivative_test="none",
    #     dual_inf_tol=1e-3,
    #     constr_viol_tol=1e-3,
    # )
    # total_time = ocp.dec_var_operator.get_var(
    #     solution, group="states", name="time", stage=-1
    # )
    # print(f"Maneuver time {total_time:.3f} sec")

# timing (note: this could be rather unstable due to the VM and resources available)
# with 2500 intervals, per model algebra NLP call (con + jac + hess)
# casadi:            : 1.35sec
# JAX CPU            : 3.25sec (and 34.8sec for 25000 intervals, linear scaling)
# JAX GPU K80 (64bit): 0.12sec (and 0.89sec for 25000 intervals, still sublinear scaling)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
main()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
DEBUG:lumos.models.tires.utils:FILE_TYPE is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:FILE_FORMAT is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:LENGTH is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:FORCE is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:ANGLE is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:MASS is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:TIME is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:TYRESIDE is not a numeric value and is discarded.
INFO:lumos.models.tracks:left distance violation: 0
INFO:lumos.models.tracks:right distance violation: 0
INFO:lumos.optimal_control.scaled_mesh_ocp:Triggering jax JIT
INFO:lumos.optimal_control.nlp:time.objective: 0.000032
INFO:lumos.optimal_control.nlp:time.gradient: 0.000249
INFO:lumos.optimal_control.nlp:tim

starting the first solve!


INFO:lumos.optimal_control.nlp:model_algebra.constraints: 2.246174
INFO:lumos.optimal_control.nlp:model_algebra.jacobian: 7.413820
INFO:lumos.optimal_control.nlp:model_algebra.hessian: 14.216124
INFO:lumos.optimal_control.nlp:continuity.constraints: 0.005842
INFO:lumos.optimal_control.nlp:continuity.jacobian: 0.009149
INFO:lumos.optimal_control.nlp:continuity.hessian: 0.000127
INFO:lumos.optimal_control.nlp:cyclic.constraints: 0.000483
INFO:lumos.optimal_control.nlp:cyclic.jacobian: 0.000021
INFO:lumos.optimal_control.nlp:cyclic.hessian: 0.000013
INFO:lumos.optimal_control.scaled_mesh_ocp:Triggering jax JIT completed
INFO:lumos.optimal_control.scaled_mesh_ocp:Time NLP execution
INFO:lumos.optimal_control.nlp:time.objective: 0.000010
INFO:lumos.optimal_control.nlp:time.gradient: 0.000362
INFO:lumos.optimal_control.nlp:time.hessian: 0.000002
INFO:lumos.optimal_control.nlp:inputs_penalty.objective: 0.000774
INFO:lumos.optimal_control.nlp:inputs_penalty.gradient: 0.002009
INFO:lumos.optima

b'Algorithm terminated successfully at a locally optimal point, satisfying the convergence tolerances (can be specified by options).'
finished in 129 iterations
Maneuver time 114.964 sec


## Visualiza results
TODO: add some nice plots/animations